# Project Part 3: Final Presentation

### CS-GY 6083 Principals of Database Systems - Spring 2024

* **Author**: [Nicola Maiorana]
* **Date**: [2024-04-14]
* **Email**: [nam10102@nyu.edu]
* **Class**: [CS-GY 6083]
---

## Overview of Presentation
- Project Overview
- IDE And Presentation Tools
- Code
    - ER Diagram
    - Tables DDL (8 Tables)
    - Views DDL (3 Views)
    - Procedures DDL (2 Procedures)
    - Functions DDL (2 Functions)
    - Triggers DDL (2 Triggers)
    - Business Objects (8 Classes)
- Forms
    - Table Inserts
    - Table Deletes
    - Selects
- Reports
    - Total Sales
    - Total Units Sold
---

## Project Overview
- MySQL Database
- Python
- 2-Tier Architecture
- Record Album Information:
    - Album Name, Recording Date, Artists, Record Label, Genre, Sales Information
    - Supporting Tables:
        - Artist, Genre, Record Label, Album, Track, Sales Tables
- Business Objects:
    - RecordArtist, RecordGenre, RecordLabel, RecordAlbum, RecordTrack, RecordSale, GroupMember, MembersToArtists
    - CRUD Operations:
        - Create, Read, Update, Delete
    - Helper Operations:
        - Read All, Read By Id, Read By Name, Add Member, Remove Member, ...

- Forms Demonstration:
    - Record Album Manager
    - Record Tracks Manager

---

In [1]:
import tools.db_utils as dbu
import tools.mermaid_diagrams as md

### Import required modules

- tools.db_utils: Contains the database connection and query functions
- tools.mermaid_diagrams: Allows for the creation of Mermaid diagrams

## Code

### ER Diagram
- Normalization
- Integrity Enforcement Used (Where/Why)
- Isolation Level Used (Where/Why)

In [2]:
md.mm(md.er_diargram)

---
## Tables DDL
- Record Artists
- Group Members
- Members To Artists
- Record Genres
- Record Labels
- Record Albums
- Record Tracks
- Record Sales
```sql
CREATE TABLE RECORD_ARTISTS (
    artist_id int NOT NULL AUTO_INCREMENT,
    artist_name varchar(40),
    PRIMARY KEY (artist_id)
);

CREATE TABLE GROUP_MEMBERS (
    member_id int NOT NULL AUTO_INCREMENT,
    member_name varchar(40),
    member_country varchar(40),
    member_birthdate date,
    PRIMARY KEY (member_id)
);

CREATE TABLE MEMBERS_TO_ARTISTS (
    members_to_artists_id int NOT NULL AUTO_INCREMENT,
    member_id int,
    artist_id int,
    member_from_date date,
    member_to_date date,
    PRIMARY KEY (members_to_artists_id),
    FOREIGN KEY (member_id) REFERENCES GROUP_MEMBERS(member_id),
    FOREIGN KEY (artist_id) REFERENCES RECORD_ARTISTS(artist_id)
);

CREATE TABLE RECORD_GENRES (
    genre_id int NOT NULL AUTO_INCREMENT,
    genre_name varchar(40),
    genre_description varchar(255),
    PRIMARY KEY (genre_id),
    UNIQUE (genre_name)
);

CREATE TABLE RECORD_LABELS (
    record_label_id int NOT NULL AUTO_INCREMENT,
    record_label_name varchar(50),
    PRIMARY KEY (record_label_id),
    UNIQUE (record_label_name)
);

CREATE TABLE RECORD_ALBUMS (
    album_id int NOT NULL AUTO_INCREMENT,
    album_name varchar(50),
    release_date date,
    artist_id int,
    genre_id int,
    record_label_id int,
    PRIMARY KEY (album_id),
    FOREIGN KEY (artist_id) REFERENCES RECORD_ARTISTS(artist_id),
    FOREIGN KEY (genre_id) REFERENCES RECORD_GENRES(genre_id),
    FOREIGN KEY (record_label_id) REFERENCES RECORD_LABELS(record_label_id),
    UNIQUE (album_name, release_date)
);

CREATE TABLE RECORD_TRACKS (
    track_id int NOT NULL AUTO_INCREMENT,
    album_id int,
    track_name varchar(40),
    track_number int,
    genre_id int,
    PRIMARY KEY (track_id),
    FOREIGN KEY (album_id) REFERENCES RECORD_ALBUMS(album_id),
    FOREIGN KEY (genre_id) REFERENCES RECORD_GENRES(genre_id),
    UNIQUE (album_id, track_number)
);

CREATE TABLE RECORD_SALES (
    sale_id int NOT NULL AUTO_INCREMENT,
    album_id int,
    sale_date date,
    sale_quantity int,
    unit_sale_price decimal,
    PRIMARY KEY (sale_id),
    FOREIGN KEY (album_id) REFERENCES RECORD_ALBUMS(album_id)
);
```

### Database Utility Functions

- Query DB and return a pandas DataFrame

In [ ]:
import tools.db_utils as dbu
pandas_index_settings = ['album_id']
display(dbu.query_to_df(('select '
                            'rec.album_id, '
                            'rec.album_name, '
                            'ra.artist_name, '
                            'rg.genre_name, '
                            'rg.genre_description, '
                            'rl.record_label_name '
                            'from record_albums rec '
                            'JOIN record_artists ra ON ra.artist_id = rec.artist_id '
                            'JOIN record_genres rg ON rg.genre_id = rec.genre_id '
                            'JOIN record_labels rl ON rl.record_label_id = rec.record_label_id'), pandas_index_settings))

___
### Views DDL
- Album Information
- Album Information Details
- Band Members

#### Album Information View
```sql
CREATE VIEW album_information AS
    SELECT
        RECORD_ALBUMS.album_name,
        RECORD_ARTISTS.artist_name,
        RECORD_GENRES.genre_name,
        RECORD_LABELS.record_label_name,
        RECORD_ALBUMS.release_date
    FROM
        RECORD_ALBUMS
    JOIN RECORD_ARTISTS ON RECORD_ALBUMS.artist_id = RECORD_ARTISTS.artist_id
    JOIN RECORD_LABELS ON RECORD_ALBUMS.record_label_id = RECORD_LABELS.record_label_id
    JOIN RECORD_GENRES ON RECORD_ALBUMS.genre_id = RECORD_GENRES.genre_id;
```

In [ ]:
import tools.db_utils as dbu
pandas_index_settings = ['album_name']
display(dbu.query_to_df('select * from album_information order by release_date', pandas_index_settings))

#### Album Information Details View
```sql
CREATE VIEW album_information_details AS
    SELECT
        RECORD_ALBUMS.album_name,
        RECORD_ARTISTS.artist_name,
        record_genres.genre_name,
        RECORD_LABELS.record_label_name,
        RECORD_TRACKS.track_number,
        RECORD_TRACKS.track_name,
        RECORD_ALBUMS.release_date
    FROM
        RECORD_ALBUMS
    JOIN RECORD_ARTISTS ON RECORD_ALBUMS.artist_id = RECORD_ARTISTS.artist_id
    JOIN RECORD_LABELS ON RECORD_ALBUMS.record_label_id = RECORD_LABELS.record_label_id
    JOIN RECORD_TRACKS ON RECORD_ALBUMS.album_id = RECORD_TRACKS.album_id
    JOIN RECORD_GENRES ON RECORD_TRACKS.genre_id = RECORD_GENRES.genre_id
ORDER BY
    RECORD_ALBUMS.album_name, RECORD_TRACKS.track_number;
```

In [ ]:
import tools.db_utils as dbu
pandas_index_settings = ['album_name', 'release_date', 'artist_name', 'record_label_name', 'track_number']
display(dbu.query_to_df('select * from album_information_details order by release_date', pandas_index_settings))

#### Band Members View
```sql
CREATE VIEW band_members AS
    SELECT
        RECORD_ARTISTS.artist_name,
        GROUP_MEMBERS.member_name,
        GROUP_MEMBERS.member_country,
        MEMBERS_TO_ARTISTS.member_from_date,
        MEMBERS_TO_ARTISTS.member_to_date
    FROM
        RECORD_ARTISTS
    JOIN MEMBERS_TO_ARTISTS ON RECORD_ARTISTS.artist_id = MEMBERS_TO_ARTISTS.artist_id
    JOIN GROUP_MEMBERS ON MEMBERS_TO_ARTISTS.member_id = GROUP_MEMBERS.member_id;
```

In [ ]:
import tools.db_utils as dbu
pandas_index_settings = ['artist_name', 'member_name']
display(dbu.query_to_df('select * from band_members order by artist_name, member_name', pandas_index_settings))

___
### Procedures DDL
- Count Record Sales
- Total Record Sales


### Procedure to count the number of sales for a given album
```
DELIMITER //
CREATE PROCEDURE count_record_sales(IN album_id INT, OUT sales_count INT)
BEGIN
	SET @album_id = album_id;
    SELECT 
        COUNT(sale_id) INTO sales_count
    FROM RECORD_SALES
    WHERE RECORD_SALES.album_id = @album_id;

END //
DELIMITER ;
```


In [ ]:
import tools.db_utils as dbu
from business_objects.record_album_sql import RecordAlbum
from business_objects.record_sales_sql import RecordSales
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        record_album = RecordAlbum.read_by_name("Boston")
        all_sales = RecordSales.read_by_album_id(record_album.album_id)
        print(f'Number of sales for album {record_album.album_name} using select statement    : {len(all_sales)}')
        result = cur.callproc('count_record_sales', [record_album.album_id, 0])[1]
        print(f'Number of sales for album {record_album.album_name} using the stored procedure: {result}')

### Procedure to get the total sales for all albums
```sql
DELIMITER //
CREATE PROCEDURE total_record_sales(IN album_id INT, OUT total_sales NUMERIC)
BEGIN
	SET @album_id = album_id;
    SELECT 
        sum(sale_quantity * unit_sale_price) INTO total_sales
    FROM RECORD_SALES
    WHERE RECORD_SALES.album_id = @album_id;

END //
DELIMITER ;
```

In [ ]:
import tools.db_utils as dbu
from business_objects.record_album_sql import RecordAlbum
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        record_album = RecordAlbum.read_by_name("Boston")
        total_sales_query = dbu.query_to_df(f"select sum(sale_quantity * unit_sale_price) as total_sales from record_sales where album_id = {record_album.album_id}")["total_sales"][0]
        print(f'Total sales for album {record_album.album_name} using select statement    : {total_sales_query}')
        result = float(cur.callproc('total_record_sales', (record_album.album_id, 0))[1])
        print(f'Total sales for album {record_album.album_name} using the stored procedure: {result}')

___
### Function DLL
- Album Summary Function to return a summary for an album to include the artist, producer, release date and tracks
```sql
DELIMITER //
CREATE FUNCTION album_summary (album_id int) RETURNS varchar(600)
	READS SQL DATA
BEGIN
	DECLARE done int default false;
    DECLARE track_summary varchar(400);
	DECLARE album_summary varchar(200);
    DECLARE response_summary varchar(600);
	DECLARE result_cursor CURSOR FOR
		SELECT
			concat(" ", t.track_number, "-", t.track_name) AS track_summary
			FROM record_tracks t
			WHERE t.album_id = album_id
			ORDER BY t.track_number;
	DECLARE CONTINUE HANDLER FOR not found SET done = true;
	SET album_summary = "";
    SET response_summary = "";
	OPEN result_cursor;
	read_loop: LOOP
		FETCH result_cursor INTO track_summary;
        IF done THEN
			LEAVE read_loop;
		END IF;
        SET response_summary = CONCAT(response_summary, track_summary, "\n ");
    END LOOP;
    CLOSE result_cursor;
	SELECT
		concat(
			r.album_name, "\n ",
			"Perfomed by: ", a.artist_name, "\n ",
            "Produced by: ", l.record_label_name, "\n ",
			"Released   : ", r.release_date, "\n "
            ) INTO album_summary
		FROM record_albums r, record_artists a, record_labels l
		WHERE r.album_id = album_id and a.artist_id = r.artist_id and l.record_label_id = r.record_label_id;
	SET response_summary = CONCAT(album_summary, response_summary);
    RETURN response_summary;
END //
DELIMITER ;
```

In [ ]:
import tools.db_utils as dbu
from business_objects.record_album_sql import RecordAlbum
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        record_album = RecordAlbum.read_by_name("Boston")
        cur.execute(f'select album_summary(%s)', (record_album.album_id,))
        result = cur.fetchone()[0]
        print(f'Album summary:\n {result}')

### Function to show all the members of a band over time

```sql
# Function to summarize an band members for an artist
DROP FUNCTION IF EXISTS band_summary;
DELIMITER //
CREATE FUNCTION band_summary (artist_id int) RETURNS varchar(600)
	READS SQL DATA
BEGIN
	DECLARE done int default false;
    DECLARE member_summary varchar(400);
	DECLARE band_summary varchar(200);
    DECLARE response_summary varchar(600);
	DECLARE result_cursor CURSOR FOR
		SELECT
			concat(" ", group_members.member_name,
				   " (", group_members.member_country, ")", 
				   " from: ",  members_to_artists.member_from_date, 
				   " till: ", members_to_artists.member_to_date) AS member_summary
			FROM
				record_artists
			JOIN members_to_artists ON record_artists.artist_id = members_to_artists.artist_id
			JOIN group_members ON members_to_artists.member_id = group_members.member_id
			WHERE record_artists.artist_id = artist_id;
	DECLARE CONTINUE HANDLER FOR not found SET done = true;
	SET band_summary = "";
    SET response_summary = "";
	OPEN result_cursor;
	read_loop: LOOP
		FETCH result_cursor INTO member_summary;
        IF done THEN
			LEAVE read_loop;
		END IF;
        SET response_summary = CONCAT(response_summary, member_summary, "\n ");
    END LOOP;
    CLOSE result_cursor;
	SELECT
		concat(a.artist_name, " - Band Members: \n ") INTO band_summary
		FROM record_artists a
		WHERE a.artist_id = artist_id;
	SET response_summary = CONCAT(band_summary, response_summary);
    RETURN response_summary;
END //
DELIMITER ;
```

In [ ]:
import tools.db_utils as dbu
from business_objects.record_artists_sql import RecordArtist
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        artist_id = 1
        record_artist = RecordArtist.read_by_name("Pink Floyd")
        cur.execute(f'select band_summary(%s)', (record_artist.artist_id,))
        result = cur.fetchone()[0]
        print(f'Summary for {record_artist.artist_name}:\n{result}')

___
### Trigger DDL
- Cleanup Artists
- Cleanup Tracks

### Trigger to cleanup members_to_artists and albums when an artist is deleted
```sql
DROP TRIGGER IF EXISTS cleanup_artists;

DELIMITER //
CREATE TRIGGER cleanup_artists BEFORE DELETE ON record_artists
FOR EACH ROW
BEGIN
    DELETE FROM members_to_artists m WHERE m.artist_id = OLD.artist_id;
    UPDATE record_albums r set artist_id = null WHERE r.artist_id = OLD.artist_id;
END //
DELIMITER ;
```

In [ ]:
import tools.db_utils as dbu
from business_objects.record_artists_sql import RecordArtist
from business_objects.members_to_artists_sql import MembersToArtists
pandas_index_settings = ['artist_name', 'member_name']

with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        record_artist = RecordArtist.read_by_name("Boston")
        if record_artist is None:
            print(f'Artist Boston not found. Init data...')
            
        else:
            all_members = MembersToArtists.read_members(record_artist.artist_id)
            print(f'Number of members for artist {record_artist.artist_name} using business object    : {len(all_members)}')
            print(f'Deleting artist {record_artist.artist_name}...')
            cur.execute('delete from record_artists where artist_name = "Boston"')
            conn.commit()
            all_members = MembersToArtists.read_members(record_artist.artist_id)
            print(f'Number of members for artist {record_artist.artist_name} using business object    : {len(all_members)}')

### Trigger to cleanup tracks and sales when an album is deleted
```sql
DROP TRIGGER IF EXISTS cleanup_tracks;

DELIMITER //
CREATE TRIGGER cleanup_tracks BEFORE DELETE ON record_albums
FOR EACH ROW
BEGIN
    DELETE FROM record_tracks t WHERE t.album_id = OLD.album_id;
    DELETE FROM record_sales s WHERE s.album_id = OLD.album_id;
END //
DELIMITER ;
```

In [ ]:
import tools.db_utils as dbu
from business_objects.record_album_sql import RecordAlbum
from business_objects.record_tracks_sql import RecordTrack
from business_objects.record_sales_sql import RecordSales

pandas_index_settings = ['album_name', 'release_date', 'artist_name', 'record_label_name', 'track_number']
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        record_album = RecordAlbum.read_by_name("Boston")
        if record_album is None:
            print(f'Album Boston not found. Init data...')
            
        else:
            record_tracks = RecordTrack.read_by_album_id(record_album.album_id)
            record_sales = RecordSales.read_by_album_id(record_album.album_id)
            print(f'Number of tracks for album {record_album.album_name} using business object    : {len(record_tracks)}')
            print(f'Number of sales for album {record_album.album_name} using business object     : {len(record_sales)}')
            print(f'Deleting album {record_album.album_name}...')
            cur.execute('delete from record_albums where album_name = "Boston"')
            conn.commit()
            record_tracks = RecordTrack.read_by_album_id(record_album.album_id)
            record_sales = RecordSales.read_by_album_id(record_album.album_id)
            print(f'Number of tracks for album {record_album.album_name} using business object    : {len(record_tracks)}')
            print(f'Number of sales for album {record_album.album_name} using business object     : {len(record_sales)}')

---
## Reports

- Album Total Sales
- Album Annual Sales

In [ ]:
import tools.db_utils as dbu
pandas_index_settings = ['Album Name']
print(f'Total Sales')
total_sales_df = dbu.query_to_df(('SELECT '
                            'ra.album_name as "Album Name", '
                            'FORMAT(SUM(rs.sale_quantity), 0) as "Units Sold", '
                            'CONCAT("$", FORMAT(SUM(rs.sale_quantity * rs.unit_sale_price), 0, "en_US")) as "Total Sales" '
                        'FROM record_sales rs '
                        'JOIN record_albums ra ON ra.album_id = rs.album_id '
                        'GROUP BY ra.album_name '
                        'ORDER BY ra.album_name'), pandas_index_settings)
total_sales_df.align(['Units Sold', 'Total Sales'], )
display(total_sales_df)

pandas_index_settings = ['Album Name', 'Year']
print(f'Annual Sales')
annual_sales_df = dbu.query_to_df(('SELECT '
                            'ra.album_name as "Album Name", '
                            'YEAR(rs.sale_date) as "Year", '
                            'FORMAT(SUM(rs.sale_quantity), 0) as "Units Sold", '
                            'CONCAT("$", FORMAT(SUM(rs.sale_quantity * rs.unit_sale_price), 0, "en_US")) as "Sales" '
                        'FROM record_sales rs '
                        'JOIN record_albums ra ON ra.album_id = rs.album_id '
                        'GROUP BY YEAR(rs.sale_date), rs.album_id '
                        'ORDER BY ra.album_name'), pandas_index_settings)
display(annual_sales_df)

___
## Forms Demonstration
- Record Album Manager
- Record Tracks Manager

In [3]:
from ui_objects.record_album_manager_ui import RecordAlbumUI

RecordAlbumUI()

TypeError: unsupported operand type(s) for |: 'str' and 'NoneType'

### Table Inserts

In [ ]:
import tools.db_utils as dbu
from business_objects.record_genres_sql import RecordGenre

# Setup
test_genre_name = 'TEST Genre'
RecordGenre.delete_by_name(test_genre_name)
# Insert
new_record = RecordGenre.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
# Cleanup
RecordGenre.delete_by_name(test_genre_name)

### Table Reads

In [ ]:
from business_objects.record_album_sql import RecordAlbum
print(f'Read all albumns')
for album in RecordAlbum.read_all()[:5]:
    print(album)
    
print(f'\n\nRead by id: 5')
print(RecordAlbum.read(1))
print(f'\n\nRead by name: The Dark Side of the Moon')
print(RecordAlbum.read_by_name('The Dark Side of the Moon'))

### Table Updates

In [ ]:
from business_objects.record_genres_sql import RecordGenre

# Setup
test_genre_name = 'TEST Genre'
updated_genre_name = 'Updated Genre'
RecordGenre.delete_by_name(test_genre_name)
RecordGenre.delete_by_name(updated_genre_name)
new_record = RecordGenre.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
# Update
new_record.genre_name = updated_genre_name
new_record.genre_description = 'An updated genre'
print(f'Record to update: {new_record}')
new_record.update()
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{updated_genre_name}'",  'genre_id'))
# Cleanup
RecordGenre.delete_by_name(test_genre_name)
RecordGenre.delete_by_name(updated_genre_name)

### Table Deletes

In [ ]:
from business_objects.record_genres_sql import RecordGenre

# Setup
test_genre_name = 'TEST Genre'
RecordGenre.delete_by_name(test_genre_name)
new_record = RecordGenre.create(genre_name=test_genre_name, genre_description='A new genre')
print(f'Created record: {new_record.genre_id}')
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
# Delete by id
new_record.delete()
print(f'After delete by id: {new_record.genre_id}')
display(dbu.query_to_df(f"select * from record_genres where genre_id = '{new_record.genre_id}'",  'genre_id'))
# Delete by name
new_record = RecordGenre.create(genre_name=test_genre_name, genre_description='A new genre')
print(f'Created record: {new_record.genre_id}')
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
RecordGenre.delete_by_name(new_record.genre_name)
print(f'After delete by name: {new_record.genre_name}')
display(dbu.query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))